Code: ETL processing 
Type: Snapshot loading.
Framework:-
    Two dataframes for ingestion, transformation and load.
    df_file is used to load the data from excel file generated by COVID-19.py file generator.
    df_table_t is used to  generate the COVID-State-Daily-Report.xlsx file upon which charts are being made with Tableau.

In [3]:
import pyodbc #pyodbc is library used to connect to MSSQL
import pandas as pd #pandas for dataframe and transformation.
from datetime import datetime,date #for time and date logics

In [14]:
time_now=datetime.now()
time_format=str(time_now)[:-7]
Curr_dt=str(date.today())


In [15]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=IDEA-PC;"
                      "Database=IDW;"
                      "Trusted_Connection=yes;")

In [16]:
metadata_df= pd.read_sql_query("select * from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='COVID_T'",cnxn)

In [17]:
df_file=pd.read_excel('COVID-19-{}.xlsx'.format(Curr_dt),squeeze=True) #df from file

In [18]:
df_file['CURR_IN']="1"
df_file['DW_LOAD_TS']=time_format

In [19]:
df_file # final checking df before ingesting data in table.

,Date,States,Counts,CURR_IN,DW_LOAD_TS
0,2020-03-25,Kerala,109,1,2020-03-25 19:29:13
1,2020-03-25,Punjab,29,1,2020-03-25 19:29:13
2,2020-03-25,Delhi,30,1,2020-03-25 19:29:13
3,2020-03-25,Jammu and Kashmir,11,1,2020-03-25 19:29:13
4,2020-03-25,Ladakh,13,1,2020-03-25 19:29:13
5,2020-03-25,Rajasthan,36,1,2020-03-25 19:29:13
6,2020-03-25,Uttar Pradesh,38,1,2020-03-25 19:29:13
7,2020-03-25,Maharashtra,116,1,2020-03-25 19:29:13
8,2020-03-25,Karnataka,51,1,2020-03-25 19:29:13
9,2020-03-25,Tamil Nadu,23,1,2020-03-25 19:29:13


In [21]:
# code to insert data from dataframe to MSSQL.

for indx,row in df_file.iterrows():
    with cnxn.cursor() as crsr:
        crsr.execute("INSERT INTO IDW.dbo.COVID_T(DW_EFF_DT,STATES,RECORD_CNT,CURR_IN,DW_LOAD_TS) values(?,?,?,?,?)",row["Date"],row["States"],row["Counts"],row["CURR_IN"],row["DW_LOAD_TS"])

In [22]:
df_table_t=pd.read_sql_query("Select * from IDW.dbo.COVID_T where curr_in='1'",cnxn)

In [24]:
#this code is to generate final excel sheet which would be ingested to tableau
df_table_t.to_excel('COVID-final-report.xlsx',index=False)

In [25]:
crsr.commit()
crsr.close()
cnxn.close()